In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.signal as signal
import math
import matplotlib.image as mpimg
from multiprocessing import Pool, cpu_count

In [11]:
## Input 
#image g 


#Image = np.random.random((128,128)) #read(fname)
g = np.random.random((512,512))
#D = 
D = g.shape[0] #128
N = D**2
K = 100 # iter number
Lambda = 0.1 # thresholding 
r = 16 # kernel size
r_2 = r**2 
num_filter = r_2

## Frame: convolution kernel
A = np.zeros((num_filter,r,r))  #2D r by r
A_flat_mat = np.zeros((r_2,num_filter))
G = np.zeros((N,r,r))  #?  r*r patches of g, from conv with a 
G_flat_mat = np.zeros((r_2,N))
G_flat_mat_T= np.zeros((N,r_2))
V_coded = np.zeros((num_filter,D,D))
#V_coded_flat = np.zeros((N,num_filter))
V_flat_mat = np.zeros((num_filter,N)) # V in paper
Vc_flat_mat_T = np.zeros((num_filter,N))  #with T effect, V in paper
#GTV = np.zeros((r_2,r_2)) #
VGT = np.zeros((num_filter,r_2)) #  num_filter,r_2


In [14]:
def W_synthesis_parallel(args):
    i, A, V = args
    return signal.convolve2d(V[i, :, :], A[i, :, :], 'same', 'wrap')

def W_synthesis(A, V, g ,pool_num=8):
    num_filter = A.shape[0]
    D = g.shape[0]
    g_denoise = np.zeros((D, D))
    pool = Pool(pool_num)  # Create a process pool with the number of available CPUs
    results = pool.map(W_synthesis_parallel, [(i, A, V) for i in range(num_filter)])  # Parallelize the loop
    pool.close()
    pool.join()
    g_denoise = np.sum(results, axis=0)

    return g_denoise

In [12]:
def W_synthesis(A,V,g): #WTg
    num_filter = A.shape[0]
    D = g.shape[0]
    g_denoise = np.zeros((D,D))
    for i in range(num_filter):
        g_denoise += signal.convolve2d(V[i,:,:],A[i,:,:],'same','wrap')
    
    return g_denoise


In [13]:
%timeit -r 1 -n 1 W_synthesis(A, V_coded, g)

38.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [8]:
for poor_num in [4,8,16,32,44]:
    %timeit -r 1 -n 1 W_synthesis(A, V_coded, g)

485 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [15]:
cpu_count()

44